In [21]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [22]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [23]:
len(words)

32033

In [24]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [25]:
#dataset

block_size = 3
X, Y = [], []
for w in words[:5]:

    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] 

X = torch.tensor(X)
Y = torch.tensor(Y)


emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [26]:
X.shape, X.dtype, Y.shape, Y.dtype 

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [27]:
C = torch.randn((27,2))

In [28]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [29]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [ ]:
# torch.cat(torch.unbind(emb, 1), 1).shape
# emb.view(32, 6).shape

torch.Size([32, 6])

In [36]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1)

In [37]:
h

tensor([[-0.9339,  1.0000, -0.9828,  ...,  0.9856, -0.9211,  0.3218],
        [ 0.8230,  0.9999,  0.0252,  ...,  0.8734, -0.9166,  0.9928],
        [-0.9932,  1.0000, -0.9902,  ...,  0.5757,  0.4517,  0.8162],
        ...,
        [ 0.5095, -0.6429, -0.3588,  ..., -0.2645,  0.2412, -0.2929],
        [ 0.8988,  0.9429, -0.9994,  ...,  0.5834,  0.9324, -0.0915],
        [ 0.9577, -0.5320,  0.9986,  ..., -0.8774, -0.9590,  0.8542]])

In [38]:
h.shape

torch.Size([32, 100])

In [39]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [40]:
logits = h @ W2 + b2

In [41]:
logits.shape

torch.Size([32, 27])

In [42]:
counts = logits.exp()

In [43]:
prob = counts/ counts.sum(1, keepdim=True)

In [44]:
prob.shape

torch.Size([32, 27])

In [47]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(19.3648)

## Formatiing


In [48]:
X.shape, Y.shape

(torch.Size([32, 3]), torch.Size([32]))

In [51]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [52]:
sum(p.nelement() for p in parameters)

3481

In [53]:
emb = C[X]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.7697)